In [1]:
from sklearn.model_selection import GroupKFold
from sklearn.linear_model import SGDClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import make_pipeline
from sklearn import svm
from scipy import signal

import matplotlib.pyplot as plt
import scipy.io as sio
! pip install neurokit2
import neurokit2 as nk
import seaborn as sns

import pandas as pd
import numpy as np
import time

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.2 MB 5.3 MB/s 


In [2]:
def feat_extract_ECG(raw):
    data_ECG = {}
    for participant in range(0, 23):
        for video in range(0, 18):
            # load raw baseline and stimuli data for left and right
            basl_left = (raw["DREAMER"][0, 0]["Data"]
                         [0, participant]["ECG"][0, 0]
                         ["baseline"][0, 0][video, 0][:, 0])
            stim_left = (raw["DREAMER"][0, 0]["Data"]
                         [0, participant]["ECG"][0, 0]
                         ["stimuli"][0, 0][video, 0][:, 0])
            basl_right = (raw["DREAMER"][0, 0]["Data"]
                          [0, participant]["ECG"][0, 0]
                          ["baseline"][0, 0][video, 0][:, 1])
            stim_right = (raw["DREAMER"][0, 0]["Data"]
                          [0, participant]["ECG"][0, 0]
                          ["stimuli"][0, 0][video, 0][:, 1])
            # process with neurokit
            signals_b_l, info_b_l = nk.ecg_process(basl_left,
                                                   sampling_rate=256)
            signals_s_l, info_s_l = nk.ecg_process(stim_left,
                                                   sampling_rate=256)
            signals_b_r, info_b_r = nk.ecg_process(basl_right,
                                                   sampling_rate=256)
            signals_s_r, info_s_r = nk.ecg_process(stim_right,
                                                   sampling_rate=256)
            # divide stimuli features by baseline features
            # would be interesting to compare classification accuracy when we
            # don't do this
            features_ecg_l = nk.ecg_intervalrelated(signals_s_l) / nk.ecg_intervalrelated(signals_b_l)
            features_ecg_r = nk.ecg_intervalrelated(signals_s_r) / nk.ecg_intervalrelated(signals_b_r)
            # average left and right features
            # would be interesting to compare classification accuracy when we
            # rather include both left and right features
            features_ecg = (features_ecg_l + features_ecg_r)/2
            if not len(data_ECG):
                data_ECG = features_ecg
            else:
                data_ECG = pd.concat([data_ECG, features_ecg],
                                     ignore_index=True)
    return data_ECG

In [7]:
def preprocess_EEG(raw, feature):
    overall = signal.firwin(9, [0.0625, 0.46875], window="hamming")
    theta = signal.firwin(9, [0.0625, 0.125], window="hamming")
    alpha = signal.firwin(9, [0.125, 0.203125], window="hamming")
    beta = signal.firwin(9, [0.203125, 0.46875], window="hamming")
    filt_data = signal.filtfilt(overall, 1, raw)
    filt_theta = signal.filtfilt(theta, 1, filt_data)
    filt_alpha = signal.filtfilt(alpha, 1, filt_data)
    filt_beta = signal.filtfilt(beta, 1, filt_data)
    ftheta, psdtheta = signal.welch(filt_theta, nperseg=128)
    falpha, psdalpha = signal.welch(filt_alpha, nperseg = 128)
    fbeta, psdbeta = signal.welch(filt_beta, nperseg=128)
    feature.append(max(psdtheta))
    feature.append(max(psdalpha))
    feature.append(max(psdbeta))
    return feature

In [8]:
def feat_extract_EEG(raw):
    EEG_tmp = np.zeros((23, 18, 42))
    for participant in range(0, 23):
        for video in range(0, 18):
            for i in range(0, 14):
                B, S = [], []
                basl = (raw["DREAMER"][0, 0]["Data"]
                        [0, participant]["EEG"][0, 0]
                        ["baseline"][0, 0][video, 0][:, i])
                stim = (raw["DREAMER"][0, 0]["Data"]
                        [0, participant]["EEG"][0, 0]
                        ["stimuli"][0, 0][video, 0][:, i])
                B = preprocess_EEG(basl, B)
                S = preprocess_EEG(stim, S)
                Extrod = np.divide(S, B)
                EEG_tmp[participant, video, 3*i] = Extrod[0]
                EEG_tmp[participant, video, 3*i+1] = Extrod[1]
                EEG_tmp[participant, video, 3*i+2] = Extrod[2]
    col = []
    for i in range(0, 14):
        col.append("psdtheta_"+str(i + 1)+"_un")
        col.append("psdalpha_"+str(i + 1)+"_un")
        col.append("psdbeta_"+str(i + 1)+"_un")
    data_EEG = pd.DataFrame(EEG_tmp.reshape((23 * 18,
                                             EEG_tmp.shape[2])), columns=col)
    scaler = StandardScaler()
    for i in range(len(col)):
        data_EEG[col[i][:-3]] = scaler.fit_transform(data_EEG[[col[i]]])
    data_EEG.drop(col, axis=1, inplace=True)
    return data_EEG

In [9]:
def participant_affective(raw):
    a = np.zeros((23, 18, 9), dtype=object)
    for participant in range(0, 23):
        for video in range(0, 18):
            a[participant, video, 0] = (raw["DREAMER"][0, 0]["Data"]
                                        [0, participant]["Age"][0][0][0])
            a[participant, video, 1] = (raw["DREAMER"][0, 0]["Data"]
                                        [0, participant]["Gender"][0][0][0])
            a[participant, video, 2] = int(participant+1)
            a[participant, video, 3] = int(video+1)
            a[participant, video, 4] = ["Searching for Bobby Fischer",
                                        "D.O.A.", "The Hangover", "The Ring",
                                        "300", "National Lampoon\'s VanWilder",
                                        "Wall-E", "Crash", "My Girl",
                                        "The Fly", "Pride and Prejudice",
                                        "Modern Times", "Remember the Titans",
                                        "Gentlemans Agreement", "Psycho",
                                        "The Bourne Identitiy",
                                        "The Shawshank Redemption",
                                        "The Departed"][video]
            a[participant, video, 5] = ["calmness", "surprise", "amusement",
                                        "fear", "excitement", "disgust",
                                        "happiness", "anger", "sadness",
                                        "disgust", "calmness", "amusement",
                                        "happiness", "anger", "fear",
                                        "excitement", "sadness",
                                        "surprise"][video]
            a[participant, video, 6] = int(raw["DREAMER"][0, 0]["Data"]
                                           [0, participant]["ScoreValence"]
                                           [0, 0][video, 0])
            a[participant, video, 7] = int(raw["DREAMER"][0, 0]["Data"]
                                           [0, participant]["ScoreArousal"]
                                           [0, 0][video, 0])
            a[participant, video, 8] = int(raw["DREAMER"][0, 0]["Data"]
                                           [0, participant]["ScoreDominance"]
                                           [0, 0][video, 0])
    b = pd.DataFrame(a.reshape((23*18, a.shape[2])),
                     columns=["age", "gender", "participant",
                              "video", "video_name", "target_emotion",
                              "valence", "arousal", "dominance"])
    return b

In [12]:
#path = "DREAMER.mat"
#raw = sio.loadmat(path)
from google.colab import drive
drive.mount('/content/drive')
raw = sio.loadmat("/content/drive/MyDrive/DREAMER.mat")


df_EEG = feat_extract_EEG(raw)
df_ECG = feat_extract_ECG(raw)
df_features = pd.concat([df_EEG, df_ECG], axis=1)
df_participant_affective = participant_affective(raw)
# not really clear to me why I have to do this
# because I thought these were already integers
# but whatever
df_participant_affective["valence"] = (df_participant_affective["valence"].astype(int))
df_participant_affective["arousal"] = (df_participant_affective["arousal"].astype(int))
df_participant_affective["dominance"] = (df_participant_affective["dominance"].astype(int))
df = pd.concat([df_features, df_participant_affective], axis=1)

/usr/local/lib/python3.7/dist-packages/neurokit2/hrv/hrv_nonlinear.py:446: NeuroKitWarning: DFA_alpha2 related indices will not be calculated. The maximum duration of the windows provided for the long-term correlation is smaller than the minimum duration of windows. Refer to the `scale` argument in `nk.fractal_dfa()` for more information.
  category=NeuroKitWarning,
/usr/local/lib/python3.7/dist-packages/neurokit2/hrv/hrv_nonlinear.py:446: NeuroKitWarning: DFA_alpha2 related indices will not be calculated. The maximum duration of the windows provided for the long-term correlation is smaller than the minimum duration of windows. Refer to the `scale` argument in `nk.fractal_dfa()` for more information.
  category=NeuroKitWarning,
/usr/local/lib/python3.7/dist-packages/neurokit2/hrv/hrv_nonlinear.py:446: NeuroKitWarning: DFA_alpha2 related indices will not be calculated. The maximum duration of the windows provided for the long-term correlation is smaller than the minimum duration of wind

In [13]:
df = pd.concat([df_EEG, df_participant_affective], axis=1)

In [16]:
data = df
data = data.loc[(data['target_emotion'] == 'anger') |
              (data['target_emotion'] == 'fear') |
              (data['target_emotion'] == 'calmness')].copy()

In [17]:
data['stress_bin'] = data['target_emotion'].map(
    {'anger': 1, 'fear': 1, 'calmness': 0}
)

In [18]:
data2 = df.loc[(df['target_emotion'] == 'anger') |
                (df['target_emotion'] == 'fear') |
                (df['target_emotion'] == 'calmness') |
                (df['target_emotion'] == 'surprise') |
                (df['target_emotion'] == 'excitement') |
                (df['target_emotion'] == 'amusement') |
                (df['target_emotion'] == 'happiness') |
                (df['target_emotion'] == 'sadness') |
                (df['target_emotion'] == 'disgust')].copy()

d={'surprise': 0, 'excitement': 0, 'amusement': 0, 'happiness': 0, 'fear': 1, 'anger': 1, 'sadness':2, 'disgust':2 ,'calmness': 3}
data2['class'] = data2.target_emotion.map(d)

e={0: 0, 1: 0, 2: 0, 3: 1, 4: 1, 5: 1 }
data2['valencehigh'] = data2.valence.map(e)

f={0: 0, 1: 0, 2: 0, 3: 1, 4: 1, 5: 1 }
data2['arousalhigh'] = data2.arousal.map(e)
f={0: 0, 1: 0, 2: 0, 3: 1, 4: 1, 5: 1 }
data2['dominancehigh'] = data2.dominance.map(e)
data2.to_csv('data_afterpreprocess.csv')

In [19]:
print(data.isnull().any())

psdtheta_1        False
psdalpha_1        False
psdbeta_1         False
psdtheta_2        False
psdalpha_2        False
psdbeta_2         False
psdtheta_3        False
psdalpha_3        False
psdbeta_3         False
psdtheta_4        False
psdalpha_4        False
psdbeta_4         False
psdtheta_5        False
psdalpha_5        False
psdbeta_5         False
psdtheta_6        False
psdalpha_6        False
psdbeta_6         False
psdtheta_7        False
psdalpha_7        False
psdbeta_7         False
psdtheta_8        False
psdalpha_8        False
psdbeta_8         False
psdtheta_9        False
psdalpha_9        False
psdbeta_9         False
psdtheta_10       False
psdalpha_10       False
psdbeta_10        False
psdtheta_11       False
psdalpha_11       False
psdbeta_11        False
psdtheta_12       False
psdalpha_12       False
psdbeta_12        False
psdtheta_13       False
psdalpha_13       False
psdbeta_13        False
psdtheta_14       False
psdalpha_14       False
psdbeta_14      

In [20]:
data.to_csv('ml_data_EEGonly.csv')

In [39]:
data = pd.read_csv("./data_afterpreprocess.csv")
data.replace([np.inf, -np.inf], np.nan, inplace=True)

In [40]:
import torch
import torch.nn as nn
import torch.utils.data as Data

class LSTM(nn.Module):
    def __init__(self, input_size=132, hidden_layer_size=10, output_size=1):
       
        super().__init__()
        self.hidden_layer_size = hidden_layer_size
        self.lstm = nn.LSTM(input_size, hidden_layer_size)
        self.linear = nn.Linear(hidden_layer_size, output_size)
        self.sigmoid = nn.Sigmoid()

    def forward(self, input_x):
        input_x = input_x.view(len(input_x), 1, -1)
        hidden_cell = (torch.zeros(1, 1, self.hidden_layer_size),  # shape: (n_layers, batch, hidden_size)
                       torch.zeros(1, 1, self.hidden_layer_size))
        lstm_out, (h_n, h_c) = self.lstm(input_x, hidden_cell)
        linear_out = self.linear(lstm_out.view(len(input_x), -1))  # =self.linear(lstm_out[:, -1, :])
        predictions = self.sigmoid(linear_out)
        return predictions


In [41]:
group_kfold = GroupKFold(n_splits=10)
X = np.array(data.loc[:, 'psdtheta_1':'psdbeta_3'])

X = X.astype(np.float64)
X = np.nan_to_num(X)
y = np.array(data['arousalhigh'])
groups = np.array(data['participant'])
for train_index, test_index in group_kfold.split(X, y, groups):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

In [42]:
print(X.shape,X_train.shape,X_test.shape)

(414, 9) (378, 9) (36, 9)


In [43]:
import numpy as np

    
x, y = torch.from_numpy(X_train).to(torch.float32), torch.from_numpy(y_train).to(torch.float32)

train_loader = Data.DataLoader(
        dataset=Data.TensorDataset(x, y),  
        batch_size=1,  
        shuffle=True,  
        num_workers=2,  
    )
lstm = LSTM()  
loss_function = nn.BCELoss()  # loss
optimizer = torch.optim.Adam(lstm.parameters(), lr=0.001)  
epochs = 100
lstm.train()
for i in range(epochs):
    for seq, labels in train_loader:
        optimizer.zero_grad()
        y_pred = lstm(seq).squeeze()  
        labels = labels.squeeze()
        single_loss = loss_function(y_pred, labels)
            
        single_loss.backward()
        optimizer.step()
       
lstm.eval()

for seq, labels in train_loader:  
    y_pred = lstm(seq).squeeze()  
    labels = labels.squeeze()
    single_loss = loss_function(y_pred, labels)
#    acc = sum(y_pred==labels)/len(y_pred)
#    print("EVAL Step:", i, " accuracy: ", acc)


RuntimeError: ignored

In [35]:
from sklearn.metrics import f1_score
x, y = torch.from_numpy(X_test).to(torch.float32), torch.from_numpy(y_test).to(torch.float32)
y_test_pred = lstm(x).squeeze()
final = [] 
for i in y_test_pred:
  if i >0.5:
    final.append(1)
  else:
    final.append(0)
f1 = f1_score(y_test, final)
print(sum(final==y_test)/len(y_test),f1)

RuntimeError: ignored

In [ ]:
import torch.nn.functional as F
import torch.optim as optim
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.l1 = torch.nn.Linear(132, 64)
        self.l2 = torch.nn.Linear(64, 32)
        self.l3 = torch.nn.Linear(32, 16)
        self.l4 = torch.nn.Linear(16, 8)
        self.l5 = torch.nn.Linear(8, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        
        x = x.view(-1, 132) 
        x = F.relu(self.l1(x))
        x = F.relu(self.l2(x))
        x = F.relu(self.l3(x))
        x = F.relu(self.l4(x))
        
        return self.sigmoid(self.l5(x))

x, y = torch.from_numpy(X_train).to(torch.float32), torch.from_numpy(y_train).to(torch.float32)
train_loader = Data.DataLoader(
        dataset=Data.TensorDataset(x, y),  
        batch_size=1,  
        shuffle=True,  
        num_workers=2,  
    )

model = Net()

# construct loss and optimizer
loss_function = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)


epochs = 100
   
for i in range(epochs):
    for seq, labels in train_loader:
        optimizer.zero_grad()
        y_pred = model(seq).squeeze()  
        labels = labels.squeeze()
        single_loss = loss_function(y_pred, labels)
        single_loss.backward()
        optimizer.step()



x, y = torch.from_numpy(X_test).to(torch.float32), torch.from_numpy(y_test).to(torch.float32)
y_test_pred = model(x).squeeze()
final = [] 
for i in y_test_pred:
  if i >0.5:
    final.append(1)
  else:
    final.append(0)
f1 = f1_score(y_test, final)
print(sum(final==y_test)/len(y_test),f1)

In [ ]:
from sklearn.metrics import f1_score
def run_clf(clf):
    cv = GroupKFold(n_splits=10)
    score = []
    runtime = []
    for fold, (train, test) in enumerate(cv.split(X, y, groups)):
        clf.fit(X[train], y[train])
        start = time.time()
        score.append(clf.score(X_test, y_test))
        runtime.append(time.time() - start)
        f1 = f1_score(y[test], clf.predict(X[test]))

    return score, runtime, f1

In [27]:
results = []
names = ["Nearest Neighbors", "Linear SVM",
         "RBF SVM", "Gaussian Process",
         "Decision Tree", "Random Forest",
         "Neural Net", "AdaBoost",
         "Naive Bayes"]

classifiers = [
    KNeighborsClassifier(3),
    SVC(kernel="linear", C=0.025),
    SVC(gamma=2, C=1),
    GaussianProcessClassifier(1.0 * RBF(1.0)),
    DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5,
                           n_estimators=10,
                           max_features=1),
    MLPClassifier(alpha=1,
                  max_iter=1000),
    AdaBoostClassifier(),
    GaussianNB()]

for name, classifier in zip(names, classifiers):
    clf = make_pipeline(MinMaxScaler(), classifier)
    score, runtime,f1 = run_clf(clf)
    results.append([name,
                    np.mean(score),
                    np.mean(runtime),np.mean(f1)])

NameError: ignored

In [ ]:
results_df = pd.DataFrame(results, columns=['name',
                                            'mean_score_dominance',
                                            'mean_runtime','f1_score'])
results_df.to_csv('results.csv')

In [28]:
results_df

NameError: ignored

# 新段落

# 新段落